## 講習3 --- astropy.io.fitsの基本  


FITSデータの読み書きが可能なモジュールです。  元々、pyfitsと呼ばれていたものです。astropyに吸収されてそちらで統一的に管理されるようです。  
astropy.io.fitsの使い方は、 
http://docs.astropy.org/en/stable/io/fits/index.html
が非常に参考になります。  例が多く記載されているのでわかりやすいです。  

PyRAFでもFITSデータの読み書きはできるのですが、ndarrayとしてデータを読み出しておくとより自由な処理を行うことができます。例えば、IRAFには存在しない統計アルゴリズムを適用する、あるいは、matplotlibを用いた自由度の高い可視化などです。



In [1]:
from astropy.io import fits

まずastropy.ioからfitsをimportしておきます。

### 基本のgetdataとgetheader 
まずは簡単なデータ読み出し方法。** fits.getdata( )**と **fits.getheader( )**です。  主にインタラクティブなケースで使われます。  

まずはピクセル値をgetdataでndarrayとして読み出します。

In [2]:
data = fits.getdata('./M15/set1/SUPA00322525.fits')  #  ドームフラットの生データ 

In [3]:
data

array([[25643, 24520, 22495, ...,  9979,  9992,  9992],
       [29556, 37737, 33481, ...,  9989,  9988,  9982],
       [25799, 33996, 28080, ...,  9987, 10011,  9989],
       ..., 
       [28810, 39685, 27954, ..., 10012, 10009, 10012],
       [26548, 37805, 27199, ..., 10023, 10009, 10002],
       [25827, 34235, 25216, ..., 10038, 10039, 10023]], dtype=uint16)

ndarrayとして読み出すことができます。なので、numpyの関数が使えます。  

In [4]:
import numpy as np

In [5]:
np.median(data), np.std(data), np.min(data), np.max(data)

(22766.0, 1674.3108603132118, 9595, 43823)

オーバースキャンの部分も含めて統計をとっているので、標準偏差(np.std)が大きな値になっています。  
オーバースキャンの部分を除いて統計をとってみましょう。  

ndarrayなので、**[y, x]**の順に範囲を指定します。  
**[y_start:y_end, x_start:x_end]** のように範囲をしていします。値を省略すると、「最初から」 あるいは 「最後まで」となります。左右両方を省略すると「全部」です。    

In [6]:
subdata = data[:, :2045]
np.median(subdata), np.std(subdata), np.min(subdata), np.max(subdata)

(22772.0, 476.97799941385165, 9697, 43823)

上限値と下限値を適切に設定し、3-sigmaクリッップすると標準偏差がもう少し小さくなります。  

In [7]:
xx = np.where((subdata > 20000) & (subdata < 30000))
med = np.median(subdata[xx])
std = np.std(subdata[xx])

xx = np.where((subdata > med - 3 * std) & (subdata < med + 3 * std))
med = np.median(subdata[xx])
std = np.std(subdata[xx])
print ('{} {:.2f}'.format(med, std))   #   format()関数使ってみた

22771.0 296.07


次にgetheaderを使ってFITSヘッダを読み出します。  

In [8]:
header = fits.getheader('./M15/set1/SUPA00322525.fits')

ここで、header あるいは print(header) をcellで実行すると、FITSヘッダの中身が表示されます。  
前者と後者で表示方法に違いがあります。

In [9]:
# header   # これをするとgithubでは表示が長くなるのでここではコメントアウト

最初の10項目だけを表示させます。

In [10]:
header[:10]

SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                   16 / number of bits per data pixel                  
NAXIS   =                    2 / number of data axes                            
NAXIS1  =                 2080 / length of data axis 1                          
NAXIS2  =                 4100 / length of data axis 2                          
EXTEND  =                    F / FITS dataset may contain extensions            
BZERO   =              32768.0 / offset data range to that of unsigned short    
BSCALE  =                  1.0 / default scaling factor                         
BUNIT   = 'ADU     '           / Unit of original pixel value                   
BLANK   =               -32768 / Value used for NULL pixels                     

In [11]:
header['OBJECT']

'DOMEFLAT'

このようにヘッダのキーワードを指定して、その値を取り出すことができます。

### データの加工と書き出し   

このドームフラット1枚を使って、フラットを作成しましょう。 まずは、 バイアス値がY方向に一様とする簡易版で行います。  
ここでは、IRAFを使わずnumpyを使って行います。

In [12]:
omed = np.median(data[:, 2049:])  #  オーバースキャン部分のメジアン
imed = np.median(data[:, :2045])  # 光が当たる部分のメジアン
data = ( data - omed ) / (imed - omed)  #  ゲタを引き、規格化

整数データを割り算した結果、浮動小数点数になります。  デフォルトでは64ビットになります。

In [13]:
data.dtype

dtype('float64')

無駄にファイルサイズが大きくなるので、32ビットに変更します。  

In [14]:
data = data.astype(np.float32) 

ヘッダのBITPIXは自動的に変更されます。

ヘッダのOBJECTの文字列を変更してみます。 

In [15]:
header['OBJECT'] = 'I_FLAT'

ヘッダキーワードのBLANKはデータが整数のときのみ有効です。  
それ以外のときにこの項目があると、アプリケーションによってはWarningがでます。  
ここでは削除しておきましょう。  

In [16]:
del header['BLANK']

このデータとヘッダを新しいFITSファイルに書き出します。**fits.writeto()**を使います。

In [18]:
fits.writeto('iflat5p.fits', data, header)

フラットなんて中間ファイルなので盛りだくさんなヘッダは不要ですよ、という時には、fits.writeto()の引数のheaderを省略すると、最低限必要なヘッダを勝手に作ってくれます。

In [19]:
fits.writeto('iflat5psimple.fits', data)

元のFITSファイルに上書き更新する場合には**fits.update( )**を使います。

In [20]:
data = fits.getdata('iflat5p.fits') 
data[:, 2049:] = -999999.  # オーバースキャン部の値を負の大きな値にしておく 
fits.update('iflat5p.fits', data, header)

fits.getdata( ) や fits.getheader( )、fits.writeto( )などは簡単で便利 (これらはastropy.io.fitsの中で'convinience functions'と呼ばれている) なのですが、効率の悪いことをしています。その都度にファイルのオープンとクローズをしています。  

### fits.open() を使う
一般に、プログラムコードの中でFITSファイルの読み書きをする場合には次のように、** fits.open() **でファイルを開き、**.headerメソッド**と**.dataメソッド**を使ってヘッダーとデータを読み取ります。

In [33]:
hdulist = fits.open('./M15/set1/SUPA00322525.fits')

このFITSファイルがどのような構造になっているかを、**.info()メソッド**を使って調べることができます。

In [34]:
hdulist.info()

Filename: ./M15/set1/SUPA00322525.fits
No.    Name         Type      Cards   Dimensions   Format
  0  PRIMARY     PrimaryHDU     152   (2080, 4100)   int16 (rescales to uint16)   


一つのヘッダと画像データのセットだけが含まれています。   
この場合、hdulist[0]のみにデータが含まれており、次のようにヘッダと画像データを読み取ります。  
拡張FITSファイルの場合、hdulistの要素が複数あり、hdulist[1]などと指定します。     

In [35]:
hdr = hdulist[0].header
imdata = hdulist[0].data

In [36]:
hdr[:10]

SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                   16 / number of bits per data pixel                  
NAXIS   =                    2 / number of data axes                            
NAXIS1  =                 2080 / length of data axis 1                          
NAXIS2  =                 4100 / length of data axis 2                          
EXTEND  =                    F / FITS dataset may contain extensions            
BZERO   =              32768.0 / offset data range to that of unsigned short    
BSCALE  =                  1.0 / default scaling factor                         
BUNIT   = 'ADU     '           / Unit of original pixel value                   
BLANK   =               -32768 / Value used for NULL pixels                     

上のgetheader, getdataでやったようにフラットを作成します。  

In [37]:
omed = np.median(imdata[:, 2049:])  #  オーバースキャン部分のメジアン
imed = np.median(imdata[:, :2045])  # 光が当たる部分のメジアン
imdata =  ( imdata - omed ) / ( imed - omed )   #  バイアス値を引き、規格化、
hdulist[0].data = imdata.astype(np.float32)  #  32ビットに変更してから、hdulistに格納

この方法だとヘッダから'BLANK'は自動的に削除されている。

In [38]:
hdr[:10]

SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                  -32 / number of bits per data pixel                  
NAXIS   =                    2 / number of data axes                            
NAXIS1  =                 2080 / length of data axis 1                          
NAXIS2  =                 4100 / length of data axis 2                          
EXTEND  =                    F / FITS dataset may contain extensions            
BUNIT   = 'ADU     '           / Unit of original pixel value                   
DATE-OBS= '2004-06-24'         / Observation start date  (yyyy-mm-dd)           
UT      = '15:19:35.127'       / HH:MM:SS.S typical UTC at the exposure (middle 
UT-STR  = '15:19:35.127'       / HH:MM:SS.S UTC at the start exposure time      

In [39]:
hdr['OBJECT'] = 'I_FLAT'

新しいファイルに書き出す場合には、** .writetoメソッド**を使います。 

In [40]:
hdulist.writeto('iflat5p2.fits')

ヘッダには'BLANK'はないのに、上のようなWarningが出てしまう。  

開いたファイルは最後には閉じておきましょう。

In [41]:
hdulist.close()

下で例を示しますが、fits.open()のときも、with fits.open() as f: のように、with を使ってファイルを開くことができます。  
上では、内容を見ながらインタラクティブに処理をするため、ファイルを開いたままにしておく必要がありましたが、プログラムの中で処理する場合にはwithを使うのがよいでしょう。 

読み込んだファイルに上書き更新する場合には、 
下のように、openするときにmode='update'を指定しておきます。変更がclose()のときにファイルの中身に書き込まれます。

In [42]:
f = fits.open('iflat5p2.fits', mode='update')
data = f[0].data
data[:, 2049:] = -999999.  # オーバースキャン部の値を負の大きな値にしておく 
f[0].data = data
f.close()

### バイアス値をY方向も考慮に入れる 

numpyを使うと、IRAFでやるよりもすっきり書けます。

In [43]:
with fits.open('./M15/set1/SUPA00322525.fits') as hdulist:
    hdr = hdulist[0].header
    imdata = hdulist[0].data
    
bias = np.median(imdata[:, 2049:], axis=1) # X軸に沿ってmedianを計算。biasはサイズ4100の1次元配列 
bias = bias.reshape(4100,1) # ブロードキャスティングできるように整形

imdata = imdata - bias   #  ブロードキャスティングで引き算  

med = np.median(imdata[:, :2045])

imdata = imdata / med 
imdata = imdata.astype(np.float32)

fits.writeto('iflat5by2.fits', imdata) # ヘッダは生データのものを継承しない

### 複数枚のドームフラットからフラットを作成 

IRAFを使わずに、combineします。 numpy.median(), numpy.append()を使います。  
バイアス値のY方向依存も考慮します。  

In [44]:
import glob

stack = np.empty((0, 4100, 2080))  #  空の配列を作成

for img in glob.glob('./M15/set2/SUPA003225[0-4]5.fits'):

    imdata = fits.getdata(img)
    bias = np.median(imdata[:, 2049:], axis=1)
    bias = bias.reshape(4100,1)
    
    imdata = imdata - bias
    med = np.median(imdata[:, :2045])
    imdata = imdata / med 
    
    stack = np.append(stack, imdata[np.newaxis, :], axis=0)
        
immed = np.median(stack, axis=0)    
immed = immed.astype(np.float32)

fits.writeto('iflat5m.fits', immed)

### 人工的な画像  
ピクセル値が全部ゼロの10 x 10のFITSファイルを作り、いくつかのピクセルにだけ正の値を与えてやります。

In [45]:
imdata = np.zeros((10, 10), dtype='float32')  # 全て0の10x10のndarrayを作成
imdata[0, 1] = 100. #  (x,y)=(1,0)に100を代入。xとyが反転していることに注意
imdata[4, 6] = 50. 

fits.writeto('my10x10.fits', imdata)

出来上がったFITSファイルをDS9で見てみましょう。

** 練習問題 **  
1. 二つのFITSデータの差分をとり、新しいファイルに書き出してみましょう。